In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from gensim.models import Word2Vec, Phrases

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gohw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gohw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Flatten
from keras.initializers import Constant
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
df = pd.read_csv('train.csv')

In [5]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]

In [6]:
df.loc[df['id'].isin(ids_with_target_error),'target'] = 0

# Count Vectorizer

In [7]:
countvec = feature_extraction.text.CountVectorizer()
tfidfvec = feature_extraction.text.TfidfVectorizer()

In [8]:
model = linear_model.RidgeClassifier()

In [9]:
vector_count = countvec.fit_transform(df['text']).todense()
vector_tfidf = tfidfvec.fit_transform(df['text']).todense()

In [11]:
scores = model_selection.cross_val_score(model, vector_count, df['target'], cv=5, scoring='f1')
np.mean(scores)

0.5715569538004827

In [12]:
scores = model_selection.cross_val_score(model, vector_tfidf, df['target'], cv=5, scoring='f1')
np.mean(scores)

0.6325075863982154

# Glove

In [71]:
def create_corpus(df):
    corpus = []
    for row in tqdm(df['text']):
        word = [w.lower() for w in word_tokenize(row) if ((w.isalpha()) and (w not in stop)) ]
        corpus.append(word)
    return corpus

In [72]:
corpus = create_corpus(df)

100%|████████████████████████████████████| 7613/7613 [00:01<00:00, 3880.77it/s]


In [73]:
embedding_dict = {}
with open('glove.6B/glove.6B.300d.txt','r', encoding='utf8') as f:
    for line in f:
        val = line.split()
        word = val[0]
        vec = np.array(val[1:],'float32')
        embedding_dict[word] = vec
f.close()

In [74]:
tok = Tokenizer()
tok.fit_on_texts(corpus)
seq = tok.texts_to_sequences(corpus)

In [75]:
pad = pad_sequences(seq, maxlen=30, truncating='post', padding='post')

In [76]:
word_index = tok.word_index
len(word_index)

15015

In [77]:
emb_mat = np.zeros((len(word_index)+1,300))
for word,i in word_index.items():
    emb_vec = embedding_dict.get(word)
    if emb_vec is not None:
        emb_mat[i] = emb_vec

In [78]:
emb_mat.shape

(15016, 300)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(pad,df['target'],test_size=0.2, shuffle=True)

In [80]:
model = Sequential()
embedding = Embedding(emb_mat.shape[0], emb_mat.shape[1], embeddings_initializer=Constant(emb_mat), input_length=30, trainable=True)
model.add(embedding)
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(Flatten())
#model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
opt = Adam(lr=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [81]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 30, 300)           4504800   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
Total params: 4,598,305
Trainable params: 4,598,305
Non-trainable params: 0
_________________________________________________________________


In [82]:
history = model.fit(X_train, y_train, batch_size=4, epochs=20, validation_data=(X_test, y_test), verbose=1)

Train on 6090 samples, validate on 1523 samples
Epoch 1/20


 808/6090 [==>...........................] - ETA: 4:53:09 - loss: 0.6930 - acc: 0.50 - ETA: 2:27:57 - loss: 0.6925 - acc: 0.37 - ETA: 1:39:14 - loss: 0.6925 - acc: 0.50 - ETA: 1:14:49 - loss: 0.6919 - acc: 0.43 - ETA: 1:00:16 - loss: 0.6925 - acc: 0.45 - ETA: 50:40 - loss: 0.6926 - acc: 0.4583 - ETA: 43:42 - loss: 0.6927 - acc: 0.42 - ETA: 38:30 - loss: 0.6929 - acc: 0.43 - ETA: 34:25 - loss: 0.6923 - acc: 0.50 - ETA: 31:12 - loss: 0.6926 - acc: 0.45 - ETA: 28:33 - loss: 0.6926 - acc: 0.45 - ETA: 26:21 - loss: 0.6927 - acc: 0.43 - ETA: 24:31 - loss: 0.6928 - acc: 0.44 - ETA: 22:55 - loss: 0.6928 - acc: 0.44 - ETA: 21:34 - loss: 0.6929 - acc: 0.45 - ETA: 20:21 - loss: 0.6927 - acc: 0.45 - ETA: 19:16 - loss: 0.6930 - acc: 0.44 - ETA: 18:18 - loss: 0.6930 - acc: 0.44 - ETA: 17:27 - loss: 0.6930 - acc: 0.46 - ETA: 16:39 - loss: 0.6930 - acc: 0.48 - ETA: 15:59 - loss: 0.6931 - acc: 0.48 - ETA: 15:20 - loss: 0.6930 - acc: 0.48 - ETA: 14:48 - loss: 0.6930 - acc: 0.48 - ETA: 14:16 - loss: 0.69

1624/6090 [=======>......................] - ETA: 3:27 - loss: 0.6924 - acc: 0.565 - ETA: 3:26 - loss: 0.6924 - acc: 0.567 - ETA: 3:26 - loss: 0.6924 - acc: 0.568 - ETA: 3:25 - loss: 0.6923 - acc: 0.569 - ETA: 3:25 - loss: 0.6923 - acc: 0.570 - ETA: 3:24 - loss: 0.6923 - acc: 0.569 - ETA: 3:23 - loss: 0.6923 - acc: 0.570 - ETA: 3:23 - loss: 0.6923 - acc: 0.570 - ETA: 3:22 - loss: 0.6923 - acc: 0.569 - ETA: 3:22 - loss: 0.6923 - acc: 0.568 - ETA: 3:21 - loss: 0.6923 - acc: 0.569 - ETA: 3:21 - loss: 0.6924 - acc: 0.566 - ETA: 3:20 - loss: 0.6924 - acc: 0.567 - ETA: 3:20 - loss: 0.6923 - acc: 0.568 - ETA: 3:19 - loss: 0.6923 - acc: 0.569 - ETA: 3:19 - loss: 0.6923 - acc: 0.567 - ETA: 3:18 - loss: 0.6923 - acc: 0.566 - ETA: 3:18 - loss: 0.6923 - acc: 0.567 - ETA: 3:17 - loss: 0.6923 - acc: 0.567 - ETA: 3:17 - loss: 0.6923 - acc: 0.567 - ETA: 3:16 - loss: 0.6923 - acc: 0.568 - ETA: 3:16 - loss: 0.6923 - acc: 0.568 - ETA: 3:15 - loss: 0.6923 - acc: 0.568 - ETA: 3:15 - loss: 0.6923 - acc: 0.5

2440/6090 [===========>..................] - ETA: 2:23 - loss: 0.6916 - acc: 0.578 - ETA: 2:22 - loss: 0.6916 - acc: 0.579 - ETA: 2:22 - loss: 0.6916 - acc: 0.579 - ETA: 2:22 - loss: 0.6915 - acc: 0.580 - ETA: 2:22 - loss: 0.6916 - acc: 0.579 - ETA: 2:22 - loss: 0.6916 - acc: 0.578 - ETA: 2:21 - loss: 0.6916 - acc: 0.579 - ETA: 2:21 - loss: 0.6915 - acc: 0.580 - ETA: 2:21 - loss: 0.6915 - acc: 0.580 - ETA: 2:21 - loss: 0.6915 - acc: 0.579 - ETA: 2:21 - loss: 0.6915 - acc: 0.579 - ETA: 2:20 - loss: 0.6915 - acc: 0.579 - ETA: 2:20 - loss: 0.6915 - acc: 0.580 - ETA: 2:20 - loss: 0.6915 - acc: 0.580 - ETA: 2:20 - loss: 0.6915 - acc: 0.580 - ETA: 2:20 - loss: 0.6915 - acc: 0.580 - ETA: 2:19 - loss: 0.6915 - acc: 0.581 - ETA: 2:19 - loss: 0.6914 - acc: 0.582 - ETA: 2:19 - loss: 0.6914 - acc: 0.582 - ETA: 2:19 - loss: 0.6914 - acc: 0.582 - ETA: 2:19 - loss: 0.6914 - acc: 0.582 - ETA: 2:18 - loss: 0.6914 - acc: 0.581 - ETA: 2:18 - loss: 0.6915 - acc: 0.581 - ETA: 2:18 - loss: 0.6915 - acc: 0.5

3256/6090 [===============>..............] - ETA: 1:59 - loss: 0.6910 - acc: 0.577 - ETA: 1:59 - loss: 0.6910 - acc: 0.577 - ETA: 1:59 - loss: 0.6910 - acc: 0.577 - ETA: 1:59 - loss: 0.6911 - acc: 0.576 - ETA: 1:59 - loss: 0.6911 - acc: 0.576 - ETA: 1:59 - loss: 0.6911 - acc: 0.576 - ETA: 1:59 - loss: 0.6911 - acc: 0.575 - ETA: 1:58 - loss: 0.6911 - acc: 0.576 - ETA: 1:58 - loss: 0.6911 - acc: 0.575 - ETA: 1:58 - loss: 0.6911 - acc: 0.574 - ETA: 1:58 - loss: 0.6912 - acc: 0.574 - ETA: 1:58 - loss: 0.6912 - acc: 0.573 - ETA: 1:58 - loss: 0.6912 - acc: 0.573 - ETA: 1:58 - loss: 0.6912 - acc: 0.573 - ETA: 1:58 - loss: 0.6911 - acc: 0.574 - ETA: 1:57 - loss: 0.6912 - acc: 0.573 - ETA: 1:57 - loss: 0.6912 - acc: 0.572 - ETA: 1:57 - loss: 0.6912 - acc: 0.572 - ETA: 1:57 - loss: 0.6913 - acc: 0.571 - ETA: 1:57 - loss: 0.6912 - acc: 0.571 - ETA: 1:57 - loss: 0.6912 - acc: 0.572 - ETA: 1:57 - loss: 0.6912 - acc: 0.572 - ETA: 1:56 - loss: 0.6912 - acc: 0.572 - ETA: 1:56 - loss: 0.6912 - acc: 0.5

...

3264/6090 [===============>..............] - ETA: 6:19 - loss: 0.4205 - acc: 0.825 - ETA: 6:19 - loss: 0.4201 - acc: 0.825 - ETA: 6:18 - loss: 0.4211 - acc: 0.825 - ETA: 6:18 - loss: 0.4212 - acc: 0.825 - ETA: 6:17 - loss: 0.4216 - acc: 0.825 - ETA: 6:17 - loss: 0.4232 - acc: 0.824 - ETA: 6:16 - loss: 0.4227 - acc: 0.824 - ETA: 6:16 - loss: 0.4234 - acc: 0.824 - ETA: 6:15 - loss: 0.4230 - acc: 0.824 - ETA: 6:15 - loss: 0.4225 - acc: 0.825 - ETA: 6:15 - loss: 0.4242 - acc: 0.824 - ETA: 6:14 - loss: 0.4239 - acc: 0.824 - ETA: 6:14 - loss: 0.4247 - acc: 0.824 - ETA: 6:13 - loss: 0.4244 - acc: 0.824 - ETA: 6:13 - loss: 0.4241 - acc: 0.824 - ETA: 6:12 - loss: 0.4239 - acc: 0.824 - ETA: 6:12 - loss: 0.4234 - acc: 0.825 - ETA: 6:12 - loss: 0.4243 - acc: 0.824 - ETA: 6:11 - loss: 0.4238 - acc: 0.824 - ETA: 6:11 - loss: 0.4248 - acc: 0.824 - ETA: 6:10 - loss: 0.4255 - acc: 0.823 - ETA: 6:10 - loss: 0.4255 - acc: 0.823 - ETA: 6:09 - loss: 0.4257 - acc: 0.823 - ETA: 6:09 - loss: 0.4260 - acc: 0.8

4080/6090 [===================>..........] - ETA: 4:42 - loss: 0.4244 - acc: 0.822 - ETA: 4:42 - loss: 0.4243 - acc: 0.822 - ETA: 4:41 - loss: 0.4243 - acc: 0.822 - ETA: 4:41 - loss: 0.4243 - acc: 0.822 - ETA: 4:40 - loss: 0.4243 - acc: 0.822 - ETA: 4:40 - loss: 0.4245 - acc: 0.822 - ETA: 4:40 - loss: 0.4241 - acc: 0.822 - ETA: 4:39 - loss: 0.4237 - acc: 0.822 - ETA: 4:39 - loss: 0.4234 - acc: 0.822 - ETA: 4:38 - loss: 0.4235 - acc: 0.822 - ETA: 4:38 - loss: 0.4235 - acc: 0.822 - ETA: 4:38 - loss: 0.4235 - acc: 0.822 - ETA: 4:37 - loss: 0.4238 - acc: 0.822 - ETA: 4:37 - loss: 0.4235 - acc: 0.822 - ETA: 4:36 - loss: 0.4232 - acc: 0.822 - ETA: 4:36 - loss: 0.4229 - acc: 0.822 - ETA: 4:36 - loss: 0.4226 - acc: 0.822 - ETA: 4:36 - loss: 0.4226 - acc: 0.822 - ETA: 4:35 - loss: 0.4223 - acc: 0.823 - ETA: 4:35 - loss: 0.4221 - acc: 0.823 - ETA: 4:34 - loss: 0.4219 - acc: 0.823 - ETA: 4:34 - loss: 0.4223 - acc: 0.823 - ETA: 4:34 - loss: 0.4224 - acc: 0.823 - ETA: 4:33 - loss: 0.4221 - acc: 0.8

4896/6090 [=======================>......] - ETA: 3:22 - loss: 0.4223 - acc: 0.823 - ETA: 3:22 - loss: 0.4225 - acc: 0.823 - ETA: 3:21 - loss: 0.4222 - acc: 0.823 - ETA: 3:21 - loss: 0.4229 - acc: 0.823 - ETA: 3:21 - loss: 0.4229 - acc: 0.823 - ETA: 3:20 - loss: 0.4227 - acc: 0.823 - ETA: 3:20 - loss: 0.4225 - acc: 0.823 - ETA: 3:19 - loss: 0.4229 - acc: 0.823 - ETA: 3:19 - loss: 0.4230 - acc: 0.823 - ETA: 3:19 - loss: 0.4229 - acc: 0.823 - ETA: 3:18 - loss: 0.4232 - acc: 0.823 - ETA: 3:18 - loss: 0.4229 - acc: 0.823 - ETA: 3:18 - loss: 0.4229 - acc: 0.823 - ETA: 3:17 - loss: 0.4234 - acc: 0.823 - ETA: 3:17 - loss: 0.4236 - acc: 0.822 - ETA: 3:17 - loss: 0.4237 - acc: 0.822 - ETA: 3:17 - loss: 0.4234 - acc: 0.823 - ETA: 3:16 - loss: 0.4232 - acc: 0.823 - ETA: 3:16 - loss: 0.4231 - acc: 0.823 - ETA: 3:17 - loss: 0.4230 - acc: 0.823 - ETA: 3:16 - loss: 0.4228 - acc: 0.823 - ETA: 3:16 - loss: 0.4227 - acc: 0.823 - ETA: 3:16 - loss: 0.4224 - acc: 0.823 - ETA: 3:16 - loss: 0.4225 - acc: 0.8

5724/6090 [===========================>..] - ETA: 2:04 - loss: 0.4148 - acc: 0.827 - ETA: 2:03 - loss: 0.4146 - acc: 0.827 - ETA: 2:03 - loss: 0.4146 - acc: 0.827 - ETA: 2:03 - loss: 0.4145 - acc: 0.828 - ETA: 2:02 - loss: 0.4143 - acc: 0.828 - ETA: 2:02 - loss: 0.4141 - acc: 0.828 - ETA: 2:02 - loss: 0.4139 - acc: 0.828 - ETA: 2:01 - loss: 0.4136 - acc: 0.828 - ETA: 2:01 - loss: 0.4135 - acc: 0.828 - ETA: 2:00 - loss: 0.4133 - acc: 0.828 - ETA: 2:00 - loss: 0.4132 - acc: 0.828 - ETA: 1:59 - loss: 0.4134 - acc: 0.828 - ETA: 1:59 - loss: 0.4131 - acc: 0.828 - ETA: 1:59 - loss: 0.4134 - acc: 0.828 - ETA: 1:58 - loss: 0.4132 - acc: 0.828 - ETA: 1:58 - loss: 0.4134 - acc: 0.828 - ETA: 1:57 - loss: 0.4132 - acc: 0.829 - ETA: 1:57 - loss: 0.4134 - acc: 0.828 - ETA: 1:57 - loss: 0.4132 - acc: 0.829 - ETA: 1:56 - loss: 0.4130 - acc: 0.829 - ETA: 1:56 - loss: 0.4128 - acc: 0.829 - ETA: 1:55 - loss: 0.4130 - acc: 0.829 - ETA: 1:55 - loss: 0.4130 - acc: 0.829 - ETA: 1:55 - loss: 0.4127 - acc: 0.8

6090/6090 [==============================] - ETA: 38s - loss: 0.4111 - acc: 0.83 - ETA: 37s - loss: 0.4109 - acc: 0.83 - ETA: 37s - loss: 0.4109 - acc: 0.83 - ETA: 36s - loss: 0.4112 - acc: 0.83 - ETA: 36s - loss: 0.4112 - acc: 0.83 - ETA: 36s - loss: 0.4113 - acc: 0.83 - ETA: 35s - loss: 0.4112 - acc: 0.83 - ETA: 35s - loss: 0.4116 - acc: 0.83 - ETA: 34s - loss: 0.4117 - acc: 0.83 - ETA: 34s - loss: 0.4117 - acc: 0.83 - ETA: 33s - loss: 0.4119 - acc: 0.83 - ETA: 33s - loss: 0.4121 - acc: 0.83 - ETA: 33s - loss: 0.4119 - acc: 0.83 - ETA: 32s - loss: 0.4117 - acc: 0.83 - ETA: 32s - loss: 0.4119 - acc: 0.83 - ETA: 31s - loss: 0.4126 - acc: 0.83 - ETA: 31s - loss: 0.4124 - acc: 0.83 - ETA: 30s - loss: 0.4122 - acc: 0.83 - ETA: 30s - loss: 0.4123 - acc: 0.83 - ETA: 30s - loss: 0.4125 - acc: 0.83 - ETA: 29s - loss: 0.4123 - acc: 0.83 - ETA: 29s - loss: 0.4123 - acc: 0.83 - ETA: 28s - loss: 0.4122 - acc: 0.83 - ETA: 28s - loss: 0.4119 - acc: 0.83 - ETA: 28s - loss: 0.4123 - acc: 0.83 - ETA: 

In [83]:
pred=model.predict(X_test)

In [84]:
f1_score(y_test.as_matrix(),np.round(pred).astype(int))

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.7531806615776081

# Word2Vec embedding

In [57]:
model = Word2Vec(corpus, iter=100, min_count=1, size=300, window=10)

In [58]:
len(model.wv.index2word)

15015

In [59]:
model.wv.syn0.shape

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(15015, 300)

In [60]:
len(model.wv.index2word)

15015

In [61]:
tok = Tokenizer()
tok.fit_on_texts(corpus)
seq = tok.texts_to_sequences(corpus)
pad = pad_sequences(seq, maxlen=30, truncating='post', padding='post')

In [62]:
len(model.wv.syn0[2])

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


300

In [63]:
emb_mat_new = np.zeros((len(word_index)+1,300))
for word,i in word_index.items():
    index = model.wv.index2word.index(word)
    emb_vec_new = model.wv.syn0[index]
    if emb_vec_new is not None:
        emb_mat_new[i] = emb_vec_new

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  after removing the cwd from sys.path.


In [64]:
emb_mat_new

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.3845782 , -0.0425349 , -0.04771034, ..., -0.65308386,
         0.54728538, -0.25472206],
       [ 0.7632187 ,  0.06589568,  0.64144588, ...,  1.19303143,
         0.27725032, -1.04676747],
       ...,
       [-0.03448423, -0.11742967,  0.04690425, ..., -0.05450049,
        -0.16353075,  0.0212611 ],
       [-0.00332544,  0.03047312, -0.07278416, ...,  0.02380517,
         0.08154586,  0.0302653 ],
       [-0.0154194 ,  0.01660249, -0.07989155, ...,  0.03930873,
         0.06562558,  0.01601305]])

In [66]:
model = Sequential()
embedding = Embedding(emb_mat_new.shape[0], emb_mat_new.shape[1], embeddings_initializer=Constant(emb_mat_new), input_length=30, trainable=False)
model.add(embedding)
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
#model.add(Flatten())
#model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
opt = Adam(lr=1e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [67]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 30, 300)           4504800   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 65        
Total params: 4,598,305
Trainable params: 93,505
Non-trainable params: 4,504,800
_________________________________________________________________


In [68]:
history = model.fit(X_train, y_train, batch_size=4, epochs=20, validation_data=(X_test, y_test))

Train on 6090 samples, validate on 1523 samples
Epoch 1/20


1628/6090 [=======>......................] - ETA: 1:32:21 - loss: 0.6950 - acc: 0.25 - ETA: 31:15 - loss: 0.6941 - acc: 0.4167 - ETA: 19:00 - loss: 0.6955 - acc: 0.25 - ETA: 13:47 - loss: 0.6953 - acc: 0.28 - ETA: 10:52 - loss: 0.6944 - acc: 0.38 - ETA: 9:01 - loss: 0.6949 - acc: 0.3182 - ETA: 7:44 - loss: 0.6946 - acc: 0.384 - ETA: 6:47 - loss: 0.6942 - acc: 0.416 - ETA: 6:26 - loss: 0.6942 - acc: 0.421 - ETA: 5:28 - loss: 0.6942 - acc: 0.421 - ETA: 5:01 - loss: 0.6942 - acc: 0.428 - ETA: 4:39 - loss: 0.6940 - acc: 0.434 - ETA: 4:20 - loss: 0.6939 - acc: 0.440 - ETA: 4:04 - loss: 0.6939 - acc: 0.435 - ETA: 3:50 - loss: 0.6939 - acc: 0.431 - ETA: 3:37 - loss: 0.6938 - acc: 0.435 - ETA: 3:27 - loss: 0.6939 - acc: 0.416 - ETA: 3:17 - loss: 0.6940 - acc: 0.428 - ETA: 3:09 - loss: 0.6939 - acc: 0.452 - ETA: 3:02 - loss: 0.6939 - acc: 0.448 - ETA: 2:55 - loss: 0.6938 - acc: 0.463 - ETA: 2:49 - loss: 0.6938 - acc: 0.476 - ETA: 2:43 - loss: 0.6938 - acc: 0.477 - ETA: 2:38 - loss: 0.6937 - acc

3332/6090 [===============>..............] - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 45s - loss: 0.6933 - acc: 0.52 - ETA: 44s - loss: 0.6933 - acc: 0.52 - ETA: 44s - loss: 0.6933 - acc: 0.52 - ETA: 44s - loss: 0.6933 - acc: 0.52 - ETA: 44s - loss: 0.6933 - acc: 0.53 - ETA: 44s - loss: 0.6933 - acc: 0.53 - ETA: 44s - loss: 0.6933 - acc: 0.53 - ETA: 44s - loss: 0.6932 - acc: 0.53 - ETA: 43s - loss: 0.6932 - acc: 0.53 - ETA: 43s - loss: 0.6932 - acc: 0.53 - ETA: 43s - loss: 0.6932 - acc: 0.53 - ETA: 43s - loss: 0.6932 - acc: 0.53 - ETA: 43s - loss: 0.6933 - acc: 0.53 - ETA: 43s - loss: 0.6932 - acc: 0.53 - ETA: 43s - loss: 0.6932 - acc: 0.53 - ETA: 43s - loss: 0.6932 - acc: 0.53 - ETA: 42s - loss: 0.6932 - acc: 0.53 - ETA: 42s - loss: 0.6932 - acc: 0.53 - ETA: 

5052/6090 [=======================>......] - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6929 - acc: 0.54 - ETA: 24s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6929 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 23s - loss: 0.6928 - acc: 0.54 - ETA: 22s - loss: 0.6928 - acc: 0.54 - ETA: 

6090/6090 [==============================] - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 8s - loss: 0.6925 - acc: 0.548 - ETA: 8s - loss: 0.6925 - acc: 0.548 - ETA: 8s - loss: 0.6924 - acc: 0.548 - ETA: 8s - loss: 0.6924 - acc: 0.548 - ETA: 8s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.550 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.550 - ETA: 7s - loss: 0.6924 - acc: 0.550 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.550 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 7s - loss: 0.6924 - acc: 0.549 - ETA: 

...

2516/6090 [===========>..................] - ETA: 1:00 - loss: 0.4844 - acc: 0.787 - ETA: 1:00 - loss: 0.4853 - acc: 0.786 - ETA: 59s - loss: 0.4844 - acc: 0.787 - ETA: 59s - loss: 0.4833 - acc: 0.78 - ETA: 59s - loss: 0.4828 - acc: 0.78 - ETA: 59s - loss: 0.4819 - acc: 0.78 - ETA: 59s - loss: 0.4835 - acc: 0.78 - ETA: 59s - loss: 0.4837 - acc: 0.78 - ETA: 59s - loss: 0.4839 - acc: 0.78 - ETA: 59s - loss: 0.4830 - acc: 0.78 - ETA: 59s - loss: 0.4830 - acc: 0.78 - ETA: 59s - loss: 0.4817 - acc: 0.79 - ETA: 59s - loss: 0.4808 - acc: 0.79 - ETA: 59s - loss: 0.4807 - acc: 0.79 - ETA: 58s - loss: 0.4810 - acc: 0.79 - ETA: 58s - loss: 0.4825 - acc: 0.79 - ETA: 58s - loss: 0.4810 - acc: 0.79 - ETA: 58s - loss: 0.4808 - acc: 0.79 - ETA: 58s - loss: 0.4807 - acc: 0.79 - ETA: 58s - loss: 0.4804 - acc: 0.79 - ETA: 58s - loss: 0.4822 - acc: 0.79 - ETA: 58s - loss: 0.4821 - acc: 0.79 - ETA: 57s - loss: 0.4812 - acc: 0.79 - ETA: 57s - loss: 0.4804 - acc: 0.79 - ETA: 57s - loss: 0.4794 - acc: 0.79 - 

3620/6090 [================>.............] - ETA: 45s - loss: 0.4747 - acc: 0.79 - ETA: 45s - loss: 0.4738 - acc: 0.79 - ETA: 45s - loss: 0.4744 - acc: 0.79 - ETA: 44s - loss: 0.4746 - acc: 0.79 - ETA: 44s - loss: 0.4752 - acc: 0.79 - ETA: 44s - loss: 0.4748 - acc: 0.79 - ETA: 44s - loss: 0.4750 - acc: 0.79 - ETA: 44s - loss: 0.4748 - acc: 0.79 - ETA: 44s - loss: 0.4743 - acc: 0.79 - ETA: 44s - loss: 0.4740 - acc: 0.79 - ETA: 44s - loss: 0.4738 - acc: 0.79 - ETA: 44s - loss: 0.4736 - acc: 0.79 - ETA: 44s - loss: 0.4746 - acc: 0.79 - ETA: 44s - loss: 0.4741 - acc: 0.79 - ETA: 44s - loss: 0.4741 - acc: 0.79 - ETA: 44s - loss: 0.4737 - acc: 0.79 - ETA: 43s - loss: 0.4754 - acc: 0.79 - ETA: 43s - loss: 0.4754 - acc: 0.79 - ETA: 43s - loss: 0.4757 - acc: 0.79 - ETA: 43s - loss: 0.4770 - acc: 0.79 - ETA: 43s - loss: 0.4769 - acc: 0.79 - ETA: 43s - loss: 0.4771 - acc: 0.79 - ETA: 43s - loss: 0.4777 - acc: 0.79 - ETA: 43s - loss: 0.4785 - acc: 0.79 - ETA: 43s - loss: 0.4778 - acc: 0.79 - ETA: 

4768/6090 [======================>.......] - ETA: 32s - loss: 0.4777 - acc: 0.79 - ETA: 32s - loss: 0.4772 - acc: 0.79 - ETA: 32s - loss: 0.4769 - acc: 0.79 - ETA: 32s - loss: 0.4771 - acc: 0.79 - ETA: 32s - loss: 0.4767 - acc: 0.79 - ETA: 32s - loss: 0.4765 - acc: 0.79 - ETA: 32s - loss: 0.4762 - acc: 0.79 - ETA: 32s - loss: 0.4763 - acc: 0.79 - ETA: 32s - loss: 0.4764 - acc: 0.79 - ETA: 32s - loss: 0.4761 - acc: 0.79 - ETA: 31s - loss: 0.4758 - acc: 0.79 - ETA: 31s - loss: 0.4754 - acc: 0.79 - ETA: 31s - loss: 0.4757 - acc: 0.79 - ETA: 31s - loss: 0.4753 - acc: 0.79 - ETA: 31s - loss: 0.4755 - acc: 0.79 - ETA: 31s - loss: 0.4754 - acc: 0.79 - ETA: 31s - loss: 0.4757 - acc: 0.79 - ETA: 31s - loss: 0.4757 - acc: 0.79 - ETA: 31s - loss: 0.4760 - acc: 0.79 - ETA: 31s - loss: 0.4762 - acc: 0.79 - ETA: 31s - loss: 0.4766 - acc: 0.79 - ETA: 31s - loss: 0.4766 - acc: 0.79 - ETA: 31s - loss: 0.4766 - acc: 0.79 - ETA: 31s - loss: 0.4765 - acc: 0.79 - ETA: 31s - loss: 0.4762 - acc: 0.79 - ETA: 

6060/6090 [============================>.] - ETA: 17s - loss: 0.4776 - acc: 0.79 - ETA: 17s - loss: 0.4773 - acc: 0.79 - ETA: 17s - loss: 0.4772 - acc: 0.79 - ETA: 17s - loss: 0.4774 - acc: 0.79 - ETA: 17s - loss: 0.4774 - acc: 0.79 - ETA: 17s - loss: 0.4772 - acc: 0.79 - ETA: 17s - loss: 0.4770 - acc: 0.79 - ETA: 17s - loss: 0.4769 - acc: 0.79 - ETA: 17s - loss: 0.4771 - acc: 0.79 - ETA: 16s - loss: 0.4773 - acc: 0.79 - ETA: 16s - loss: 0.4782 - acc: 0.79 - ETA: 16s - loss: 0.4781 - acc: 0.79 - ETA: 16s - loss: 0.4786 - acc: 0.79 - ETA: 16s - loss: 0.4782 - acc: 0.79 - ETA: 16s - loss: 0.4783 - acc: 0.79 - ETA: 16s - loss: 0.4780 - acc: 0.79 - ETA: 16s - loss: 0.4782 - acc: 0.79 - ETA: 16s - loss: 0.4780 - acc: 0.79 - ETA: 16s - loss: 0.4777 - acc: 0.79 - ETA: 15s - loss: 0.4783 - acc: 0.79 - ETA: 15s - loss: 0.4786 - acc: 0.79 - ETA: 15s - loss: 0.4789 - acc: 0.79 - ETA: 15s - loss: 0.4790 - acc: 0.79 - ETA: 15s - loss: 0.4790 - acc: 0.79 - ETA: 15s - loss: 0.4786 - acc: 0.79 - ETA: 

6090/6090 [==============================] - ETA: 0s - loss: 0.4821 - acc: 0.789 - ETA: 0s - loss: 0.4819 - acc: 0.789 - ETA: 0s - loss: 0.4817 - acc: 0.789 - ETA: 0s - loss: 0.4817 - acc: 0.789 - ETA: 0s - loss: 0.4817 - acc: 0.789 - ETA: 0s - loss: 0.4816 - acc: 0.789 - 82s 14ms/step - loss: 0.4819 - acc: 0.7897 - val_loss: 0.4553 - val_acc: 0.7984


In [69]:
pred=model.predict(X_test)

In [70]:
f1_score(y_test.as_matrix(),np.round(pred).astype(int))

C:\Users\gohw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.7481542247744052